# Task 1

In [1]:
# Import necessary libraries
import pandas as pd

file_path = r"/Users/lukaandrensek/Documents/gen_i/TradingChallenge_Task1/Data_SLO.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path,sep=';',parse_dates=['ProductStartUTC','ProductStartCET'])
df = df.sort_values(by='ProductStartUTC')
# Display the first few rows of the dataset

df = df.drop(columns=["ProductStartUTC"])
df = df.set_index("ProductStartCET")
df = df.drop(columns=["Temperature"])
df

,Consumption
ProductStartCET,
2023-01-01 00:00:00,836.0
2023-01-01 01:00:00,829.0
2023-01-01 02:00:00,794.0
2023-01-01 03:00:00,760.0
2023-01-01 04:00:00,739.0
...,...
2024-10-24 19:00:00,NaN
2024-10-24 20:00:00,NaN
2024-10-24 21:00:00,NaN


# Add features

In [2]:
def add_features(df):
    df["Hour"] = df.index.hour
    df["Weekday"] = df.index.weekday
    df["Month"] = df.index.month
    
    for date in df.index:
        if "11-01" in str(date) or "10-31" in str(date):
            df.loc[date, "Weekday"] = 6
            
    hours = list(range(0, 24))
    months = list(range(1, 13))
    weekdays = list(range(0, 7))
    
    df["Hour"] = pd.Categorical(df["Hour"], categories=hours)
    df["Weekday"] = pd.Categorical(df["Weekday"], categories=weekdays)
    df["Month"] = pd.Categorical(df["Month"], categories=months)
    
    
    y = df["Consumption"]
    for i in range(30, 70):
        name = f"Consumption_{i}"
        yhat = y.shift(i)
        df[name] = yhat
    
    df = pd.get_dummies(df, columns=['Weekday'], drop_first=True, dtype=int)
    df = pd.get_dummies(df, columns=['Hour'], drop_first=True, dtype=int)
    df = pd.get_dummies(df, columns=['Month'], drop_first=True, dtype=int)
    return df
df = add_features(df)

In [3]:
df

,Consumption,Consumption_30,Consumption_31,Consumption_32,Consumption_33,Consumption_34,Consumption_35,Consumption_36,Consumption_37,Consumption_38,...,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12
ProductStartCET,,,,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,836.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2023-01-01 01:00:00,829.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2023-01-01 02:00:00,794.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2023-01-01 03:00:00,760.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2023-01-01 04:00:00,739.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-24 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1153.0,...,0,0,0,0,0,0,0,1,0,0
2024-10-24 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
2024-10-24 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0


# Split the data

In [4]:
cutoff = "2024-09-01 00:00:00"

df.dropna(how="any", inplace=True)

train = df[df.index <= cutoff]
test = df[df.index > cutoff]

x_train = train.drop(columns=["Consumption"])
y_train = train["Consumption"]

x_test = test.drop(columns=["Consumption"])
y_test = test["Consumption"]

x_train


,Consumption_30,Consumption_31,Consumption_32,Consumption_33,Consumption_34,Consumption_35,Consumption_36,Consumption_37,Consumption_38,Consumption_39,...,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12
ProductStartCET,,,,,,,,,,,,,,,,,,,,,
2023-01-03 21:00:00,1022.0,972.0,970.0,974.0,994.0,986.0,995.0,966.0,896.0,820.0,...,0,0,0,0,0,0,0,0,0,0
2023-01-03 22:00:00,1111.0,1022.0,972.0,970.0,974.0,994.0,986.0,995.0,966.0,896.0,...,0,0,0,0,0,0,0,0,0,0
2023-01-03 23:00:00,1184.0,1111.0,1022.0,972.0,970.0,974.0,994.0,986.0,995.0,966.0,...,0,0,0,0,0,0,0,0,0,0
2023-01-04 00:00:00,1222.0,1184.0,1111.0,1022.0,972.0,970.0,974.0,994.0,986.0,995.0,...,0,0,0,0,0,0,0,0,0,0
2023-01-04 01:00:00,1238.0,1222.0,1184.0,1111.0,1022.0,972.0,970.0,974.0,994.0,986.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31 20:00:00,1098.0,1087.0,1131.0,1150.0,1215.0,1266.0,1417.0,1488.0,1429.0,1187.0,...,0,0,0,0,0,1,0,0,0,0
2024-08-31 21:00:00,1150.0,1098.0,1087.0,1131.0,1150.0,1215.0,1266.0,1417.0,1488.0,1429.0,...,0,0,0,0,0,1,0,0,0,0
2024-08-31 22:00:00,1289.0,1150.0,1098.0,1087.0,1131.0,1150.0,1215.0,1266.0,1417.0,1488.0,...,0,0,0,0,0,1,0,0,0,0


# Train the model

In [5]:
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)

model.fit(x_train, y_train)

GradientBoostingRegressor(max_depth=5, random_state=42)

In [6]:
yhat = model.predict(x_test)
yhat = pd.Series(data=yhat, index=x_test.index)


In [7]:
mae = (yhat - y_test).abs().mean()
mae

103.39576037467228

# Hyperparameter tuning

In [6]:
s = """
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
import pandas as pd

# Sample DataFrame with features (X) and target (y)
data = pd.DataFrame({
    'feature1': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'feature2': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'target': [15, 25, 35, 45, 55, 65, 75, 85, 95, 105]
})


gbr = GradientBoostingRegressor(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.001, 0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=gbr, 
    param_grid=param_grid, 
    cv=5, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1 
)

grid_search.fit(x_train, y_train)

# Get the best parameters and the best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Negative MSE):", grid_search.best_score_)
"""

# Forecast

In [16]:
X = pd.read_csv(r'consumption.csv', sep=";")

def f(a):
    d, m, y = a.split(".")
    return "-".join([y, m, d])
def g(a):
    a = a.replace("H", "")
    return a + ":00:00"


def correct_time(date_str):
    if "24:00:00" in date_str:
        # Convert to datetime and add one day, then set the time to "00:00:00"
        corrected_date = pd.to_datetime(date_str.replace("24:00:00", "00:00:00")) + pd.Timedelta(days=1)
        return corrected_date
    else:
        # Convert to datetime directly if no "24:00:00" issue
        return pd.to_datetime(date_str)


X["ProductStartCET"] = X["datum"].map(f) + " " + X["ura"].map(g)
X = X[["ProductStartCET", "dejanski_prevzem"]]
X["ProductStartCET"] = X["ProductStartCET"].map(correct_time)
X = X.set_index("ProductStartCET")
X = X.dropna(how="any")
X.columns = ["Consumption"]
X

,Consumption
ProductStartCET,
2024-10-21 01:00:00,1046.0
2024-10-21 02:00:00,995.0
2024-10-21 03:00:00,962.0
2024-10-21 04:00:00,997.0
2024-10-21 05:00:00,1003.0
...,...
2024-11-05 16:00:00,1397.0
2024-11-05 17:00:00,1553.0
2024-11-05 18:00:00,1647.0


In [17]:
def extend_dataframe_with_zeros(df, h):
    # Find the last timestamp in the index
    last_timestamp = df.index[-1]
    
    # Generate a new DatetimeIndex for the next 'h' hours
    future_index = pd.date_range(start=last_timestamp + pd.Timedelta(hours=1), periods=h, freq='H')
    
    # Create a DataFrame with 0.0 values for the future hours
    future_df = pd.DataFrame(0.0, index=future_index, columns=df.columns)
    
    # Append the future rows to the original DataFrame
    extended_df = pd.concat([df, future_df])
    
    return extended_df
X = extend_dataframe_with_zeros(X, 48)
X

/var/folders/k9/ftg3b6kn0nb8m_84gwwh031r0000gn/T/ipykernel_2401/4134554812.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_index = pd.date_range(start=last_timestamp + pd.Timedelta(hours=1), periods=h, freq='H')


,Consumption
2024-10-21 01:00:00,1046.0
2024-10-21 02:00:00,995.0
2024-10-21 03:00:00,962.0
2024-10-21 04:00:00,997.0
2024-10-21 05:00:00,1003.0
...,...
2024-11-07 16:00:00,0.0
2024-11-07 17:00:00,0.0
2024-11-07 18:00:00,0.0
2024-11-07 19:00:00,0.0


In [18]:
X = add_features(X)

In [19]:
X

,Consumption,Consumption_30,Consumption_31,Consumption_32,Consumption_33,Consumption_34,Consumption_35,Consumption_36,Consumption_37,Consumption_38,...,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12
2024-10-21 01:00:00,1046.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
2024-10-21 02:00:00,995.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
2024-10-21 03:00:00,962.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
2024-10-21 04:00:00,997.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
2024-10-21 05:00:00,1003.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-07 16:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2024-11-07 17:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2024-11-07 18:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2024-11-07 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0


In [20]:
x = X.drop(columns=["Consumption"])
x

,Consumption_30,Consumption_31,Consumption_32,Consumption_33,Consumption_34,Consumption_35,Consumption_36,Consumption_37,Consumption_38,Consumption_39,...,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12
2024-10-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
2024-10-21 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
2024-10-21 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
2024-10-21 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
2024-10-21 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-07 16:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2024-11-07 17:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2024-11-07 18:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2024-11-07 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0


In [21]:
x = x.dropna()
yhat = model.predict(x)
yhat = pd.Series(data=yhat, index=x.index)
yhat

2024-10-23 22:00:00    1459.217503
2024-10-23 23:00:00    1344.743605
2024-10-24 00:00:00    1181.122987
2024-10-24 01:00:00    1101.045637
2024-10-24 02:00:00    1040.029208
                          ...     
2024-11-07 16:00:00    1255.704977
2024-11-07 17:00:00    1403.096617
2024-11-07 18:00:00    1665.438332
2024-11-07 19:00:00    1646.400524
2024-11-07 20:00:00    1633.550848
Length: 359, dtype: float64

In [22]:
d1 = "2024-11-06 00:00:00"
d2 = "2024-11-06 23:00:00"
yhat = yhat[d1:d2]
yhat

2024-11-06 00:00:00    1056.581449
2024-11-06 01:00:00    1155.187963
2024-11-06 02:00:00    1123.561846
2024-11-06 03:00:00    1133.766402
2024-11-06 04:00:00    1115.830610
2024-11-06 05:00:00    1186.536754
2024-11-06 06:00:00    1341.419928
2024-11-06 07:00:00    1549.300390
2024-11-06 08:00:00    1669.937335
2024-11-06 09:00:00    1550.648753
2024-11-06 10:00:00    1288.054100
2024-11-06 11:00:00    1265.547744
2024-11-06 12:00:00    1187.419676
2024-11-06 13:00:00    1177.990180
2024-11-06 14:00:00    1231.880852
2024-11-06 15:00:00    1268.279421
2024-11-06 16:00:00    1328.199316
2024-11-06 17:00:00    1504.922035
2024-11-06 18:00:00    1645.307483
2024-11-06 19:00:00    1726.239572
2024-11-06 20:00:00    1726.661691
2024-11-06 21:00:00    1649.447580
2024-11-06 22:00:00    1496.145658
2024-11-06 23:00:00    1418.020438
dtype: float64

In [23]:
yhat


2024-11-06 00:00:00    1056.581449
2024-11-06 01:00:00    1155.187963
2024-11-06 02:00:00    1123.561846
2024-11-06 03:00:00    1133.766402
2024-11-06 04:00:00    1115.830610
2024-11-06 05:00:00    1186.536754
2024-11-06 06:00:00    1341.419928
2024-11-06 07:00:00    1549.300390
2024-11-06 08:00:00    1669.937335
2024-11-06 09:00:00    1550.648753
2024-11-06 10:00:00    1288.054100
2024-11-06 11:00:00    1265.547744
2024-11-06 12:00:00    1187.419676
2024-11-06 13:00:00    1177.990180
2024-11-06 14:00:00    1231.880852
2024-11-06 15:00:00    1268.279421
2024-11-06 16:00:00    1328.199316
2024-11-06 17:00:00    1504.922035
2024-11-06 18:00:00    1645.307483
2024-11-06 19:00:00    1726.239572
2024-11-06 20:00:00    1726.661691
2024-11-06 21:00:00    1649.447580
2024-11-06 22:00:00    1496.145658
2024-11-06 23:00:00    1418.020438
dtype: float64

In [24]:
dates = yhat.index
values = yhat.values
dates = dates.strftime('%Y-%m-%dT%H:%M:%S')
result = pd.DataFrame({
    "ProductStartCET" : dates, 
    "Forecast" : values
})
result

,ProductStartCET,Forecast
0,2024-11-06T00:00:00,1056.581449
1,2024-11-06T01:00:00,1155.187963
2,2024-11-06T02:00:00,1123.561846
3,2024-11-06T03:00:00,1133.766402
4,2024-11-06T04:00:00,1115.830610
5,2024-11-06T05:00:00,1186.536754
6,2024-11-06T06:00:00,1341.419928
7,2024-11-06T07:00:00,1549.300390
8,2024-11-06T08:00:00,1669.937335
9,2024-11-06T09:00:00,1550.648753


In [25]:
result.to_csv('11_06.csv', index=False, header=False,sep=',')